Crime Data:
    Row= parishes
    Column= Month - Year

In [ ]:
import pandas as pd


# Read the CSV file into a DataFrame
crime_Data = pd.read_csv('data/Crime Data Month Year.csv')
crime_Data.head()


School Data:



In [ ]:
import os
import pandas as pd

columns = [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024]

parishes = [
    "Acadia", "Allen", "Ascension", "Assumption", "Avoyelles", "Beauregard", "Bienville", "Bossier",
    "Caddo", "Calcasieu", "Caldwell", "Cameron", "Catahoula", "Claiborne", "Concordia", "DeSoto",
    "East Baton Rouge", "East Carroll", "East Feliciana", "Evangeline", "Franklin", "Grant", "Iberia",
    "Iberville", "Jackson", "Jefferson", "Jefferson Davis", "Lafayette", "Lafourche", "LaSalle",
    "Lincoln", "Livingston", "Madison", "Morehouse", "Natchitoches", "Orleans", "Ouachita",
    "Plaquemines", "Pointe Coupee", "Rapides", "Red River", "Richland", "Sabine", "St. Bernard",
    "St. Charles", "St. Helena", "St. James", "St. John the Baptist", "St. Landry", "St. Martin",
    "St. Mary", "St. Tammany", "Tangipahoa", "Tensas", "Terrebonne", "Union", "Vermilion", "Vernon",
    "Washington", "Webster", "West Baton Rouge", "West Carroll", "West Feliciana", "Winn"
]

folder_path = "data/School Data Year"

# Create empyty DF with column for years and rows as parishes
school_data = pd.DataFrame(index=parishes, columns=columns)

# Sort file names so years are consistent
items_in_folder = sorted(os.listdir(folder_path))

for item_name in items_in_folder:
    item_full_path = os.path.join(folder_path, item_name)

    if os.path.isfile(item_full_path):

        try:
            year = int("".join(filter(str.isdigit, item_name))[:4])
            if year not in school_data.columns:
                print(f"⚠️ {item_name} skipped — year {year} not in template")
            else:
                df = pd.read_excel(item_full_path)
                # Only waNT THE COLUMN NAMED DPS AND THE ROWS 2 AND 64
                schoolfiltered = df.loc[0:63,"DPS"]

                # add the grade number to the appropriate year column
                school_data[year] = schoolfiltered.values

        except Exception as e:
            print(f"Error reading {item_name}: {e}")

school_data.head()

Home Mortgage Rates:



In [ ]:
HomeRatesUnfiltered = pd.read_excel("data/Home Mortgage Rates.xlsx")
HomeRatesUnfiltered["observation_date"] = pd.to_datetime(HomeRatesUnfiltered["observation_date"])

# Resample first (needs datetime index)
HomeRates_Data = (
    HomeRatesUnfiltered
        .set_index("observation_date")
        .resample("M")["MORTGAGE30US"]
        .mean()
)

# Convert end-of-month dates to YYYY-MM
HomeRates_Data.index = HomeRates_Data.index.to_period("M").astype(str)
HomeRates_Data = HomeRates_Data.to_frame().T

HomeRates_Data.head()


Home Values:

In [ ]:
HomeValuesUnfilterd = pd.read_csv("data/Home Values Month Year.csv")
HomeValuesUnfilterd["RegionName"] = HomeValuesUnfilterd["RegionName"].str.replace(" Parish", "", regex=False)
HomeValuesUnfilterd = HomeValuesUnfilterd.set_index("RegionName")
HomeValues_Data = HomeValuesUnfilterd.drop(columns=["SizeRank"])

HomeValues_Data.head()





Median Household Income:

In [ ]:
import os
import pandas as pd

def simplify(col):
    if not isinstance(col, str):
        return col
    if col in ["GEO_ID", "NAME", "Geographic Area Name"]:
        return col

    parts = col.split("!!")
    last = parts[-1].strip()
    last = (last.replace("(", "")
                .replace(")", "")
                .replace(",", "")
                .replace("--", "_")
                .replace(" ", "_"))
    return last

folder_path = "data/Median Household Income"
columns = [2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]

MedianHouseIncome_Data = {}

items_in_folder = sorted(os.listdir(folder_path))

for item_name in items_in_folder:
    item_full_path = os.path.join(folder_path, item_name)

    try:
        # Year
        year = int("".join(filter(str.isdigit, item_name))[:4])
        if year not in columns:
            continue

        # Load entire CSV raw
        df_raw = pd.read_csv(item_full_path, header=None)

        # Detect header row → row containing "!!"
        header_row_index = df_raw.apply(lambda row: row.astype(str).str.contains("!!").any(), axis=1).idxmax()

        # Set header
        df_raw.columns = df_raw.iloc[header_row_index]

        # Drop all rows up to header row
        df = df_raw.drop(index=list(range(header_row_index+1)))

        df = df.reset_index(drop=True)

        # Drop GEO_ID column (first column always)
        df = df.drop(columns=df.columns[0])

        # Drop MOE columns
        df = df.drop(columns=df.filter(regex="Margin of Error").columns)

        # Clean NAME column (which is now correctly labeled)
        name_col = df.columns[0]
        df[name_col] = df[name_col].str.replace(", Louisiana", "", regex=False)
        df[name_col] = df[name_col].str.replace(" Parish", "", regex=False)

        # Set index
        df = df.set_index(name_col)

        # Simplify column names
        df.columns = [simplify(col) for col in df.columns]

        # Reindex by parish list
        df = df.reindex(parishes)

        # Store
        MedianHouseIncome_Data[year] = df

    except Exception as e:
        print(f"Error reading {item_name}: {e}")

print(MedianHouseIncome_Data)


HOme Bedroom Values


In [ ]:
import os
import pandas as pd

folder_path = "data/Home Values Roomes"   # <-- your folder

# List of Louisiana parishes (cleaned)
parishes = [
    "Acadia", "Allen", "Ascension", "Assumption", "Avoyelles", "Beauregard",
    "Bienville", "Bossier", "Caddo", "Calcasieu", "Caldwell", "Cameron",
    "Catahoula", "Claiborne", "Concordia", "DeSoto", "East Baton Rouge",
    "East Carroll", "East Feliciana", "Evangeline", "Franklin", "Grant",
    "Iberia", "Iberville", "Jackson", "Jefferson", "Jefferson Davis",
    "Lafayette", "Lafourche", "LaSalle", "Lincoln", "Livingston", "Madison",
    "Morehouse", "Natchitoches", "Orleans", "Ouachita", "Plaquemines",
    "Pointe Coupee", "Rapides", "Red River", "Richland", "Sabine",
    "St. Bernard", "St. Charles", "St. Helena", "St. James",
    "St. John the Baptist", "St. Landry", "St. Martin", "St. Mary",
    "St. Tammany", "Tangipahoa", "Tensas", "Terrebonne", "Union",
    "Vermilion", "Vernon", "Washington", "Webster", "West Baton Rouge",
    "West Carroll", "West Feliciana", "Winn"
]

# MASTER DATA STORAGE
HomeValues_LA = {}

# Loop through every file
for filename in sorted(os.listdir(folder_path)):
    if not filename.endswith(".csv"):
        continue

    filepath = os.path.join(folder_path, filename)
    print("Loading:", filename)

    df = pd.read_csv(filepath)

    # Clean region names
    df["RegionName"] = df["RegionName"].str.replace(" Parish", "", regex=False)
    df["RegionName"] = df["RegionName"].str.replace(" County", "", regex=False)

    # Filter to Louisiana
    df_LA = df[(df["StateName"] == "LA") & (df["RegionType"] == "county")]

    # Extra filter to ensure only valid parishes remain
    df_LA = df_LA[df_LA["RegionName"].isin(parishes)]

    # Drop metadata columns to keep only date columns
    metadata_cols = ["RegionID", "SizeRank", "RegionName", "RegionType", "StateName",
                     "State", "Metro", "StateCodeFIPS", "MunicipalCodeFIPS"]
    date_cols = [col for col in df_LA.columns if col not in metadata_cols]

    # Store cleaned dataframe
    HomeValues_LA[filename] = df_LA.set_index("RegionName")[date_cols]

print("Loaded Louisiana home values for:", len(HomeValues_LA), "files")




Combine Data


In [29]:
def yearly_home_values(df):
    df = df.copy()
    df = df.T
    df.index = pd.to_datetime(df.index)
    df['Year'] = df.index.year
    yearly = df.groupby('Year').mean()
    return yearly.T

# Convert home value files to yearly
HomeValues_LA_Yearly = {}
for name, df in HomeValues_LA.items():
    HomeValues_LA_Yearly[name] = yearly_home_values(df)

# Mortgage yearly
mort_yearly = (
    HomeRatesUnfiltered
        .set_index("observation_date")
        .resample("Y")["MORTGAGE30US"]
        .mean()
        .to_frame()
)
mort_yearly.index = mort_yearly.index.year
mort_yearly.rename(columns={"MORTGAGE30US": "MortgageRate"}, inplace=True)

# Crime yearly
crime = crime_Data.copy()

if crime.columns[0] == "":
    crime = crime.drop(columns=crime.columns[0])

crime_long = crime.melt(
    id_vars="Parish",
    var_name="MonthYear",
    value_name="CrimeRate"
)

crime_long["MonthYear"] = pd.to_datetime(crime_long["MonthYear"], format="%b-%y")
crime_long["Year"] = crime_long["MonthYear"].dt.year

crime_yearly = (
    crime_long.groupby(["Parish", "Year"])["CrimeRate"]
    .mean()
    .reset_index()
)

# School yearly
school_yearly = school_data.reset_index().melt(
    id_vars="index",
    var_name="Year",
    value_name="School"
)
school_yearly.rename(columns={"index": "Parish"}, inplace=True)
school_yearly["Year"] = school_yearly["Year"].astype(int)

# Income yearly
income_frames = []
for year, df in MedianHouseIncome_Data.items():
    temp = df.reset_index()
    temp = temp.iloc[:, :2]   # Parish + value
    temp.columns = ["Parish", "Income"]
    temp["Year"] = year
    income_frames.append(temp)

income_yearly = pd.concat(income_frames, ignore_index=True)

# Build base MASTER structure (1 row per Parish × Year × Bedroom)
YEARS = list(range(2014, 2025))
import itertools

master = pd.DataFrame(list(itertools.product(parishes, YEARS)), columns=["Parish", "Year"])

# 🔥 ADD BEDROOM SUPPORT: Merge ALL bedroom datasets
bedroom_dfs = []

for name, df in HomeValues_LA_Yearly.items():
    # Extract bedroom count from filename (e.g., "3Bd.csv" → 3)
    bedrooms = int(name.replace("Bd.csv", ""))

    df = df.copy()
    df.index.name = "Parish"   # Fix index name

    # Melt into long format: Parish | Year | HomeValue
    long_df = df.reset_index().melt(
        id_vars="Parish",
        var_name="Year",
        value_name="HomeValue"
    )

    # Convert Year column from string to integer
    long_df["Year"] = long_df["Year"].astype(int)

    # Add Bedrooms column
    long_df["Bedrooms"] = bedrooms

    bedroom_dfs.append(long_df)

# Combine all bedroom datasets
all_homes = pd.concat(bedroom_dfs, ignore_index=True)


# Now master will have 5× more rows once merged (1–5 bedrooms per year)
master = master.merge(all_homes, on=["Parish", "Year"], how="left")

# Merge Crime / School / Income / Mortgage
master = master.merge(crime_yearly, on=["Parish", "Year"], how="left")
master = master.merge(school_yearly, on=["Parish", "Year"], how="left")
master = master.merge(income_yearly, on=["Parish", "Year"], how="left")

mort_yearly["Year"] = mort_yearly.index
master = master.merge(mort_yearly, on="Year", how="left")





C:\Users\fresh\AppData\Local\Temp\ipykernel_41208\15421683.py:18: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("Y")["MORTGAGE30US"]


Ridding of Naan

In [30]:
num_cols = ["HomeValue", "CrimeRate", "School", "Income", "MortgageRate"]

for col in num_cols:
    master[col] = pd.to_numeric(master[col], errors="coerce")

master = master.sort_values(["Parish", "Bedrooms", "Year"])
# Compute the Louisiana statewide crime average per year
state_year_crime = (
    master.groupby("Year")["CrimeRate"].mean()
)

# Fill missing crime values with the year’s mean
master["CrimeRate"] = master.apply(
    lambda row: state_year_crime[row["Year"]] if pd.isna(row["CrimeRate"]) else row["CrimeRate"],
    axis=1
)
for col in ["CrimeRate", "School", "Income"]:
    master[col] = (
        master.groupby("Parish")[col]
              .transform(lambda s: s.interpolate().bfill().ffill())
    )
master["MortgageRate"] = (
    master["MortgageRate"]
        .interpolate()
        .bfill()
        .ffill()
)


print(master.isna().sum())
master = master.dropna(subset=["HomeValue"])
print(master.isna().sum())

import os
os.makedirs("data/processed", exist_ok=True)

master.to_csv("data/processed/master_dataset.csv", index=False)


Parish            0
Year              0
HomeValue       329
Bedrooms        231
CrimeRate         0
School            0
Income           11
MortgageRate      0
dtype: int64
Parish          0
Year            0
HomeValue       0
Bedrooms        0
CrimeRate       0
School          0
Income          0
MortgageRate    0
dtype: int64
